LOAD DATA

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load all datasets
customers = pd.read_csv('data/olist_customers_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
payments = pd.read_csv('data/olist_order_payments_dataset.csv')
reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
category_translation = pd.read_csv('data/product_category_name_translation.csv')

CLEAN AND CORRECT DATA TYPE

In [ ]:
# 1. Check missing values
customers.isnull().sum()
# 2. Convert date columns
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date', 
                'order_estimated_delivery_date']
for col in date_columns:
    orders[col] = pd.to_datetime(orders[col])

# 3. Remove cancelled orders (optional for analysis)
orders_clean = orders[orders['order_status'] == 'delivered'].copy()

# 4. Create master dataset
master_df = orders_clean.merge(customers, on='customer_id', how='left')
master_df = master_df.merge(order_items, on='order_id', how='left')
master_df = master_df.merge(products, on='product_id', how='left')
master_df = master_df.merge(payments, on='order_id', how='left')
master_df = master_df.merge(reviews, on='order_id', how='left')
master_df = master_df.merge(category_translation, on='product_category_name', how='left')

# 5. Feature Engineering
master_df['delivery_time'] = (master_df['order_delivered_customer_date'] - 
                                master_df['order_purchase_timestamp']).dt.days
master_df['estimated_delivery_time'] = (master_df['order_estimated_delivery_date'] - 
                                         master_df['order_purchase_timestamp']).dt.days
master_df['delivery_delay'] = master_df['delivery_time'] - master_df['estimated_delivery_time']
master_df['is_late'] = (master_df['delivery_delay'] > 0).astype(int)

master_df['year'] = master_df['order_purchase_timestamp'].dt.year
master_df['month'] = master_df['order_purchase_timestamp'].dt.month
master_df['day_of_week'] = master_df['order_purchase_timestamp'].dt.dayofweek
master_df['hour'] = master_df['order_purchase_timestamp'].dt.hour




df = master_df
df = df.drop(columns=["product_name_lenght","product_photos_qty","product_description_lenght","review_comment_title"])


# Convert to datetime if not already
# order_approved_at → use purchase timestamp
df['order_approved_at'].fillna(df['order_purchase_timestamp'], inplace=True)

# delivered to carrier → approved date
df['order_delivered_carrier_date'].fillna(df['order_approved_at'], inplace=True)

# delivered to customer → carrier date
df['order_delivered_customer_date'].fillna(df['order_delivered_carrier_date'], inplace=True)

# review timestamps → delivered to customer
df['review_creation_date'].fillna(df['order_delivered_customer_date'], inplace=True)
df['review_answer_timestamp'].fillna(df['review_creation_date'], inplace=True)



# Original category name
df['product_category_name'].fillna('Unknown', inplace=True)

# English category
df['product_category_name_english'].fillna('Unknown', inplace=True)

num_product_cols = [
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]

for col in num_product_cols:
    df[col].fillna(df[col].median(), inplace=True)


df['payment_sequential'].fillna(df['payment_sequential'].mode()[0], inplace=True)
df['payment_type'].fillna(df['payment_type'].mode()[0], inplace=True)
df['payment_installments'].fillna(df['payment_installments'].median(), inplace=True)
df['payment_value'].fillna(df['payment_value'].median(), inplace=True)

# Review ID
df['review_id'].fillna('No Review', inplace=True)

# Review score → median is standard
df['review_score'].fillna(df['review_score'].median(), inplace=True)

# Review message → empty string (NOT "Unknown")
df['review_comment_message'].fillna('', inplace=True)

# delivery_time & delay → fill using median
df['delivery_time'].fillna(df['delivery_time'].median(), inplace=True)
df['delivery_delay'].fillna(df['delivery_delay'].median(), inplace=True)
df = df.convert_dtypes()



# df.to_csv('outputs/cleaned_olist_data.csv', index=False)


CHECK DATA

In [4]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,review_answer_timestamp,product_category_name_english,delivery_time,estimated_delivery_time,delivery_delay,is_late,year,month,day_of_week,hour
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,housewares,8,15,-7,0,2017,10,0,10
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,housewares,8,15,-7,0,2017,10,0,10
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,housewares,8,15,-7,0,2017,10,0,10
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,2018-08-08 18:37:50,perfumery,13,19,-6,0,2018,7,1,20
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,2018-08-22 19:07:58,auto,9,26,-17,0,2018,8,2,8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115723 entries, 0 to 115722
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115723 non-null  string        
 1   customer_id                    115723 non-null  string        
 2   order_status                   115723 non-null  string        
 3   order_purchase_timestamp       115723 non-null  datetime64[ns]
 4   order_approved_at              115723 non-null  datetime64[ns]
 5   order_delivered_carrier_date   115723 non-null  datetime64[ns]
 6   order_delivered_customer_date  115723 non-null  datetime64[ns]
 7   order_estimated_delivery_date  115723 non-null  datetime64[ns]
 8   customer_unique_id             115723 non-null  string        
 9   customer_zip_code_prefix       115723 non-null  Int64         
 10  customer_city                  115723 non-null  string        
 11  

In [6]:
df.describe()

,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_zip_code_prefix,order_item_id,price,freight_value,product_weight_g,...,payment_value,review_score,delivery_time,estimated_delivery_time,delivery_delay,is_late,year,month,day_of_week,hour
count,115723,115723,115723,115723,115723,115723.0,115723.0,115723.0,115723.0,115723.0,...,115723.0,115723.0,115723.0,115723.0,115723.0,115723.0,115723.0,115723.0,115723.0,115723.0
mean,2017-12-31 09:55:44.254227712,2017-12-31 20:21:30.749004288,2018-01-03 16:47:15.734210048,2018-01-12 21:37:04.062355968,2018-01-24 06:03:00.866378752,35079.679519,1.197048,119.911096,19.9814,2105.417367,...,171.809143,4.087329,12.022087,23.474236,-11.452045,0.073183,2017.539824,6.032509,2.746446,14.759607
min,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-10-04 00:00:00,1003.0,1.0,0.85,0.0,0.0,...,0.0,1.0,0.0,2.0,-146.0,0.0,2016.0,1.0,0.0,0.0
25%,2017-09-12 18:16:08,2017-09-13 03:26:07.500000,2017-09-14 22:57:06,2017-09-22 22:35:33.500000,2017-10-04 00:00:00,11310.0,1.0,39.9,13.08,300.0,...,60.85,4.0,6.0,18.0,-17.0,0.0,2017.0,3.0,1.0,11.0
50%,2018-01-18 21:40:03,2018-01-19 09:57:24,2018-01-23 17:34:45,2018-02-01 11:22:42,2018-02-15 00:00:00,24320.0,1.0,74.9,16.28,700.0,...,108.11,5.0,10.0,23.0,-12.0,0.0,2018.0,6.0,3.0,15.0
75%,2018-05-04 10:29:14,2018-05-04 16:54:05,2018-05-07 16:31:00,2018-05-15 00:32:48,2018-05-25 00:00:00,58805.5,1.0,132.9,21.16,1800.0,...,188.94,5.0,15.0,28.0,-7.0,0.0,2018.0,8.0,4.0,19.0
max,2018-08-29 15:00:37,2018-08-29 15:10:26,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-10-25 00:00:00,99980.0,21.0,6735.0,409.68,40425.0,...,13664.08,5.0,209.0,155.0,189.0,1.0,2018.0,12.0,6.0,23.0
std,NaN,NaN,NaN,NaN,NaN,29855.201461,0.701221,182.670252,15.71829,3772.299975,...,265.639373,1.34496,9.454323,8.814688,10.171896,0.260438,0.50424,3.229684,1.961227,5.328485
